In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults

import alphashape as alph

import numpy as np
from scipy.ndimage import affine_transform

import geojson 

import makogen as mg

set_defaults(default_color="#ECF0FF", axes=False, grid=True,theme="light", axes0=True, ortho=True, transparent=True)
set_sidecar("CadQuery", init=True)

in2mm = lambda x: x*25.4
lzip = lambda *args, **kwargs: list(zip(*args, **kwargs)) 

def get_poly_verts(poly):
    verts = poly.vertices().vals()
    coords_x = [v.X for v in verts]
    coords_y = [v.Y for v in verts]
    return coords_x, coords_y
    

Overwriting auto display for cadquery Workplane and Shape


/home/otacon/anaconda3/envs/cadquery/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [141]:
from dataclasses import dataclass, asdict, field

from makogen import DefaultLayout

@dataclass
class MakoGenerator():
    def set(self, inplace=False, **kwargs):
      og_args = asdict(self)
      new_args = og_args | kwargs 
      if not inplace: # return copy 
          return self.__class__(**new_args)
      else:
        for key, value in new_args.items():
          setattr(self, key, value)
        return self

@dataclass
class StackedLeverless(MakoGenerator):
  width:float = in2mm(14)
  height:float = in2mm(6)
  depth:float = 5
  hole_diameter:float = 8
  hole_offset:float = 25
  hole_deltax:float = 275
  hole_deltay:float = 275
  fillet_radius:float = 8
  switch_mount_dims:float = (14, 14)
  keycap_dimensions:float = (19.75, 19.75)
  keycap_fillet_radius:float = .5
  layout:list[list[int]] = field(default_factory=lambda:list(DefaultLayout), hash=True)
    
  def base(self):
    base = cq.Workplane('XY').box(self.width, self.height, self.depth).edges("|Z").fillet(self.fillet_radius)
    
    width_left = self.width - self.hole_offset
    while width_left > self.hole_diameter or width_left == 0:
        base = base.faces(">Z").rect(width_left, self.height-self.hole_offset, forConstruction=True).vertices().hole(self.hole_diameter, self.depth*2)
        width_left -= self.hole_deltax
    
    height_left = self.height - self.hole_offset
    while height_left > self.hole_diameter or height_left == 0:
        base = base.faces(">Z").rect(self.width-self.hole_offset, height_left, forConstruction=True).vertices().hole(self.hole_diameter, self.depth*2)
        height_left -= self.hole_deltay
    
    return base
    
    
  def switchplate(self):
    # TODO: Error checking? Might be worth checking if the layout is in the bounda
    base = self.base()
    switch_plate = base.faces(">Z").pushPoints(self.layout).rect(*self.switch_mount_dims).cutThruAll()
    return switch_plate


  def faceplate(self):
    base = self.base()
    # keebcap_faceplate = base.faces(">Z").pushPoints(list(self.layout)).rect(*keebcap_dimensions).cutThruAll()

    s = (
    cq.Sketch()
        .push(list(self.layout))
        .rect(*self.keycap_dimensions)
        .clean()
        .reset()
        .vertices()
        .fillet(self.keycap_fillet_radius)
    )

    r = cq.Workplane().placeSketch(s).extrude(self.depth, both=True)
    keebcap_faceplate = base.cut(r)

    return keebcap_faceplate

Done, using side car 'Cadquery'


In [50]:

@dataclass()
class test():
    val1:float = 10
    val2:float = 20
    
asdict(test())

{'val1': 10, 'val2': 20}

## Bottom Plate

In [90]:
field?

Signature:
field(
    *,
    default=<dataclasses._MISSING_TYPE object at 0x79a79c7818b0>,
    default_factory=<dataclasses._MISSING_TYPE object at 0x79a79c7818b0>,
    init=True,
    repr=True,
    hash=None,
    compare=True,
    metadata=None,
)
Docstring:
Return an object to identify dataclass fields.

default is the default value of the field.  default_factory is a
0-argument function called to initialize a field's value.  If init
is True, the field will be a parameter to the class's __init__()
function.  If repr is True, the field will be included in the
object's repr().  If hash is True, the field will be included in
the object's hash().  If compare is True, the field will be used
in comparison functions.  metadata, if specified, must be a
mapping which is stored but not otherwise examined by dataclass.

It is an error to specify both default and default_factory.
File:      ~/anaconda3/envs/cadquery/lib/python3.9/dataclasses.py
Type:      function


In [91]:
# widthmm = in2mm(14)
# heightmm = in2mm(6)
# depthmm = 5
# screwhole_diameter = 8

# dowel_offset = 60
# dowel_width = 20

# bolt_offset = 25
# # bolt_deltax = (widthmm-bolt_offset)/2
# # bolt_deltay = (heightmm-bolt_offset)
# bolt_deltax = 275
# bolt_deltay = 275

# # base 
# base = cq.Workplane('XY').box(widthmm, heightmm, depthmm).edges("|Z").fillet(7)

# # dowel_slots = cq.Workplane('XY').rect(widthmm, heightmm-dowel_offset, forConstruction=True).vertices().box(dowel_width, depthmm, depthmm)
# # dowel_slots = dowel_slots.workplane().rect(widthmm-dowel_offset, heightmm, forConstruction=True).vertices().box(depthmm, dowel_width, depthmm)


# width_left = widthmm - bolt_offset
# while width_left > screwhole_diameter or width_left == 0:
#     base = base.faces(">Z").rect(width_left, heightmm-bolt_offset, forConstruction=True).vertices().hole(screwhole_diameter, depthmm*2)
#     width_left -= bolt_deltax

# height_left = heightmm - bolt_offset
# while height_left > screwhole_diameter or height_left == 0:
#     print("cutting: ", height_left)
#     base = base.faces(">Z").rect(widthmm-bolt_offset, height_left, forConstruction=True).vertices().hole(screwhole_diameter, depthmm*2)
#     height_left -= bolt_deltay
    
# base

## Hollow Plate

In [8]:
eightbutton_coords = [(-26.15, -11.75), (0,0), (26.65, -3.2), (53.3, -19.45), 
                        (-26.15, -33.75), (0, -22), (26.65, -25.2), (53.3, -41.45)]
  
# the four buttons for directionals is just the first 4 buttons reflected along the Y axis
fourbutton_coords = np.matmul(eightbutton_coords[4:], [[-1, 0], [0, 1]])
  
cstick_coords = [(0,0), (-18.5, -12.75), (0, 25.5), (-18.5, 12.75), (18.5, 12.75)]
# modifier buttons are just the first two cstick buttons reflected along the Y axis
modifier_coords = np.matmul(cstick_coords[:2], [[-1, 0], [0, 1]])
startbutton_coords = [(0,0)]
startgccmx_coords = [(-22, 0), (0,0), (22, 0)]


eightbutton_center = (100, 58)
fourbutton_center = (-100, 58)
modifier_center = (-70, -45)
cstick_center = (70, -45)
startbutton_center = (0, 18)

# for whatever reason, trying to cut everything at once causes an error,
# so we have to do it one at a time
eightbutton_layout = np.add(eightbutton_coords, eightbutton_center)  
fourbutton_layout = np.add(fourbutton_coords, fourbutton_center)
modifier_layout = np.add(modifier_coords, modifier_center)
cstick_layout = np.add(cstick_coords, cstick_center)
startbutton_layout = np.add(startbutton_coords, startbutton_center)

complete_layout = np.concatenate([eightbutton_layout, fourbutton_layout, modifier_layout, cstick_layout, startbutton_layout], axis=0)

cap_diameter = 22.5

s1 = (
    cq.Sketch()
    .push(eightbutton_layout.tolist())
    .circle(cap_diameter/2)
    .clean()
    .reset()
    .vertices()
    .fillet(.5)
)

s2 = (
    cq.Sketch()
    .push(cstick_layout.tolist())
    .circle(cap_diameter/2)
    .clean()
    .reset()
    .vertices()
    .fillet(.5)
)

s3 = (
    cq.Sketch()
    .push(modifier_layout.tolist())
    .circle(cap_diameter/2)
    .clean()
    .reset()
    .vertices()
    .fillet(.5)
)

s4 = (
    cq.Sketch()
    .push(fourbutton_layout.tolist())
    .circle(cap_diameter/2)
    .clean()
)

s5 = (
    cq.Sketch()
    .push(startbutton_layout.tolist())
    .circle(cap_diameter/2)
    .clean()
)

import sys
r = cq.Workplane().placeSketch(s1, s2, s3, s4, s5).extrude(10e3, both=True)
f1cap_faceplate = base.cut(r)
f1cap_faceplate

Done, using side car 'Cadquery'


# Keeb Cap Faceplate

In [7]:
keebcap_dimensions = (19.5, 19.5)

keebcap_faceplate = base.faces(">Z").pushPoints(complete_layout).rect(*keebcap_dimensions).cutThruAll()

s = (
    cq.Sketch()
    .push(complete_layout.tolist())
    .rect(*keebcap_dimensions)
    .clean()
    .reset()
    .vertices()
    .fillet(2)
)

r = cq.Workplane().placeSketch(s).extrude(depthmm, both=True)
keebcap_faceplate = base.cut(r)

keebcap_faceplate

Done, using side car 'Cadquery'


In [12]:
complete_layout

array([[  73.85,   46.25],
       [ 100.  ,   58.  ],
       [ 126.65,   54.8 ],
       [ 153.3 ,   38.55],
       [  73.85,   24.25],
       [ 100.  ,   36.  ],
       [ 126.65,   32.8 ],
       [ 153.3 ,   16.55],
       [ -73.85,   24.25],
       [-100.  ,   36.  ],
       [-126.65,   32.8 ],
       [-153.3 ,   16.55],
       [ -70.  ,  -45.  ],
       [ -51.5 ,  -57.75],
       [  70.  ,  -45.  ],
       [  51.5 ,  -57.75],
       [  70.  ,  -19.5 ],
       [  51.5 ,  -32.25],
       [  88.5 ,  -32.25],
       [   0.  ,   18.  ]])

# Switch Plate

In [11]:
switch_mount_dimensions = (14, 14)

switch_plate = base.faces(">Z").pushPoints(complete_layout).rect(*switch_mount_dimensions).cutThruAll()
switch_plate

NameError: name 'base' is not defined

In [9]:
modelu = cq.importers.importStep('/home/otacon/repos/Model-U/src/Model-U.step')
modelu_footprint = modelu.faces(">Z").wires("not (>X or <X)").toPending().extrude(depthmm, both=True)


In [10]:
magnet_diameter = 11

pico_width = 21
pico_height = 53

usbc_width = 19.5
usbc_height = 44
usbc_mount_width = 28
usbc_mount_height = 9

left_handrest_dim = (68, 79)
right_handrest_dim = (68, 79)
left_handrest_center = (-130, -30)
right_handrest_center = (130, -30)

left_corner_dim = (40, 30)
right_corner_dim = (40, 30)

# cadquery is making getting the bbox coords hard for all the rects, so I am  going around it
vertices = base.faces(">Z").pushPoints(complete_layout).rect(*switch_mount_dimensions).vertices().vals()
coords_x = [v.X for v in vertices]
coords_y = [v.Y for v in vertices]
minx = min(coords_x)
maxx = max(coords_x)
miny = min(coords_y)
maxy = max(coords_y)
print(minx, maxx, miny, maxy)

usbc_slot = cq.Workplane('XY').hLine(usbc_mount_width/2).vLine(-usbc_mount_height).hLine(-(usbc_mount_width-usbc_width)/2).vLine(-1*(usbc_height-usbc_mount_height)).hLineTo(0).mirrorY()
pico_slot = cq.Workplane('XY').rect(pico_width, pico_height)
gcc_cable_slot = cq.Workplane('XY').rect(13, usbc_height+5).extrude(depthmm, both=True).translate((0, heightmm/2, 0))
modelu_slot = modelu_footprint.translate((0,heightmm/2-5, 0))

# # extrude 
# usbc_poly = usbc_slot.offset2D(1, kind="intersection").extrude(depthmm, both=True).translate((40, heightmm/2-1, 0))
# cavity = cq.Workplane("XY").rect(maxx-minx, maxy-miny).extrude(depthmm, both=True).translate((0, 0))
# left_handrest =  cq.Workplane("XY").rect(*left_handrest_dim).extrude(depthmm, both=True).translate(left_handrest_center)
# right_handrest =  cq.Workplane("XY").rect(*right_handrest_dim).extrude(depthmm, both=True).translate(right_handrest_center)
# left_corner = cq.Workplane("XY").rect(*left_corner_dim).extrude(depthmm, both=True).translate((minx, maxy))
# right_corner = cq.Workplane("XY").rect(*right_corner_dim).extrude(depthmm, both=True).translate((maxx, maxy))


# cavity = cavity.cut(left_handrest).cut(right_handrest).cut(left_corner).cut(right_corner)
# wiring_cavity = base.cut(gcc_cable_slot).cut(modelu_slot).cut(cavity)
# wiring_cavity

-160.3 160.3 -64.75 65.0


In [11]:
usbc_slot = cq.Workplane('XY').hLine(usbc_mount_width/2).vLine(-usbc_mount_height).hLine(-(usbc_mount_width-usbc_width)/2).vLine(-1*(usbc_height-usbc_mount_height)).hLineTo(0).mirrorY()
usbc_poly = usbc_slot.offset2D(.1, kind="intersection").extrude(depthmm, both=True).translate((40, heightmm/2-1, 0))
usbc_poly

Done, using side car 'Cadquery'


In [12]:
modelu = cq.importers.importStep('/home/otacon/repos/Model-U/src/Model-U.step')
modelu_footprint = modelu.faces(">Z").wires("not (>X or <X)").toPending().extrude(depthmm, both=True)


In [13]:
modelu.faces(">Z").wires("not (>X or <X)")

Done, using side car 'Cadquery'


In [14]:
# slot = cq.Workplane("XY")

# show(modelu.translate((0, 6.75)), slot)

In [21]:
def create_base(width=in2mm(14), height=in2mm(6), depth = 5, hole_diameter=8, hole_offset = 25, hole_deltax=275, hole_deltay=275, fillet_radius=5):
  """
  create_base _summary_

  _extended_summary_

  Parameters
  ----------
  width : _type_, optional
      _description_, by default in2mm(14.5)
  height : _type_, optional
      _description_, by default in2mm(7)
  depth : int, optional
      _description_, by default 5
  hole_diameter : int, optional
      _description_, by default 8
  hole_offset : int, optional
      _description_, by default 25
  hole_deltax : int, optional
      _description_, by default 300
  hole_deltay : int, optional
      _description_, by default 300
  fillet_radius : int, optional
      _description_, by default 5

  Returns
  -------
  _type_
      _description_
  """
  #dowel_offset = 60
  #dowel_width = 20
  
  # based on 
  center_hole_spacing = 50 

  # base 
  base = cq.Workplane('XY').box(width, height, depth).edges("|Z").fillet(fillet_radius)

  width_left = width - hole_offset
  while width_left > hole_diameter or width_left == 0:
      base = base.faces(">Z").rect(width_left, height-hole_offset, forConstruction=True).vertices().hole(hole_diameter, depth*2)
      width_left -= hole_deltax
  
  height_left = height - hole_offset
  while height_left > hole_diameter or height_left == 0:
      print("cutting: ", height_left)
      base = base.faces(">Z").rect(width-hole_offset, height_left, forConstruction=True).vertices().hole(hole_diameter, depth*2)
      height_left -= hole_deltay

  return base


def make_wiring_layer(cable_xoffet:float=0, usbc_offset:float=1, usbc_buffer_width:float=40, usbc_buffer_height:float=28.5, cavity):
    cavity_offset = 20
    bolt_cover_width = 20
    bolt_cover_height = 20

    newbase = create_base()
    cavity_poly = cq.Workplane('XY').box(widthmm-cavity_offset, heightmm-cavity_offset, depthmm)

    corners = cq.Workplane("XY")
    width_left = widthmm - bolt_offset
    while width_left > screwhole_diameter or width_left == 0:
        corners = corners.rect(width_left, heightmm-bolt_offset, forConstruction=True).vertices().box(bolt_cover_width, bolt_cover_height, depthmm)
        width_left -= bolt_deltax

# height_left = heightmm - bolt_offset
# while height_left > screwhole_diameter or height_left == 0:
#     print("cutting: ", height_left)
#     base = base.faces(">Z").rect(widthmm-bolt_offset, height_left, forConstruction=True).vertices().hole(screwhole_diameter, depthmm*2)
#     height_left -= bolt_deltay

modelu_slot = (
    cq.Sketch()
    .rect(20+1, 13.5+1)
    .edges(">Y")
    .clean().reset()
    .vertices(">Y")
    .fillet(5)
    .clean()
)

modelu_slot = cq.Workplane("XY").placeSketch(modelu_slot).extrude(depthmm/2, both=True)
modelu_slot = modelu_slot.translate((cable_xoffset, heightmm/2-((13.5+1)/2)-usbc_depth))
cable_buffer = cq.Workplane("XY").box(usbc_buffer_width, usbc_buffer_height, depthmm).translate((cable_xoffset, heightmm/2-((13.5+1)/2)-usbc_depth))
# modelu_slot = modelu_slot.translate((cable_xoffset, heightmm/2-((13.5+1)/2)-usbc_depth))

gcc_cable_slot = cq.Workplane('XY').rect(13, usbc_buffer_height).extrude(depthmm/2, both=True).translate((cable_xoffset, heightmm/2-((13.5+1)/2)-usbc_depth))

wiring_cavity = base.cut(cavity_poly.cut(corners)).edges("|Z").fillet(2)
wiring_cavity = wiring_cavity.union(base.intersect(cable_buffer))

try:
    wiring_cavity = wiring_cavity.cut(gcc_cable_slot).edges("|Z").fillet(1)
except Exception as e:
    wiring_cavity = wiring_cavity.cut(gcc_cable_slot)

wiring_cavity = wiring_cavity.cut(modelu_slot)

show(wiring_cavity)

cutting:  127.39999999999998
Done, using side car 'Cadquery'


In [16]:
# magplate = base.faces(">Z").rect(maxx-minx-magnet_diameter, maxy-miny-magnet_diameter, forConstruction=True).vertices().circle(screwhole_diameter/2)

# magplate = magplate.cutThruAll()
# magplate

In [17]:
cardboardmx = PartGroup(
    [
        # Part(cq.importers.importsvg('/mnt/chromeos/MyFiles/Downloads/3_SwitchPlate_v2.svg').rotateAboutCenter((0,0,1), 180).translate((223.1, -181.25, 0)) ),
        Part(base, "bottom"),
        # Part(wiring_cavity.translate((0,0,depthmm*1)), "pcb_holder2"),
        Part(wiring_cavity.translate((0,0,depthmm*1)), "pcb_holder", color="black"),
        Part(switch_plate.translate((0,0,depthmm*2)), "switchplate2", color="black"),
        Part(f1cap_faceplate.translate((0,0,depthmm*3)), "faceplate1"),
        # Part(dowel.translate((widthmm/2-dowel_width/4,heightmm/2-dowel_offset/2, 10)), "dowel")

    ],
    "carboardMX"
)

show(cardboardmx, default_color="blue")

Done, using side car 'Cadquery'


In [ ]:
#### from cadquery import exporters

exporters.export(base.edges(">Z"), "artifacts/bottom.dxf")
exporters.export(wiring_cavity.edges(">Z"), "artifacts/pcb_layer_1.dxf")
# exporters.export(wiring_cavity_last_layer.edges(">Z"), "artifacts/pcb_layer_2.dxf")
exporters.export(switch_plate.edges(">Z"), "artifacts/switch_plate.dxf")
exporters.export(keebcap_faceplate.edges(">Z"), "artifacts/keebcap_faceplate.dxf")
exporters.export(f1cap_faceplate.edges(">Z"), "artifacts/f1_cap_faceplate.dxf")
# exporters.export(dowel.edges(">Z"), "artifacts/dowel.dxf")

In [ ]:
# exporters.export(cardboardmx, "artifacts/controller.svg")

In [1022]:
# dir(cardboardmx)